# Visualize Images

In [1]:
import numpy as np
from PIL import Image
from glob import glob
from tqdm import tqdm
from os.path import join as osj
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import UnidentifiedImageError
from collections import OrderedDict, Counter
from utils import read_json, save_json


DATA_DIR = '../../../Datasets/Visual_Genome'

### Load Data (attr + reg)

In [2]:
data_attr = read_json(osj(DATA_DIR, './attributes.json'))

In [3]:
_p1 = glob(osj(DATA_DIR, 'images_1', '*.jpg'))
_p2 = glob(osj(DATA_DIR, 'images_2', '*.jpg'))

im_paths = _p1 + _p2

#### Image Dims

In [4]:
_corrupt = []

def _read_im_size(p: str) -> int:
    try:
        im = Image.open(p)
        w, h = im.size
    except UnidentifiedImageError:
        _corrupt.append(p)
        w, h = 800, 600
    return w

_sizes = [_read_im_size(path) for path in tqdm(im_paths)]

_ = int(np.mean(_sizes))
__ = int(np.std(_sizes))

f"Image Width: µ = {_}, σ = {__}"

100%|██████████| 108246/108246 [00:37<00:00, 2887.53it/s]


'Image Width: µ = 500, σ = 133'

In [5]:
def _path2id(p: str) -> int:
    p = p.split('/')[-1]
    p = p.split('.')[0]
    p = int(p)
    return p

id2paths = {_path2id(path): path for path in im_paths
            if path not in _corrupt}

#### Plot Functions

In [6]:
def _annotate(ax, box: list, label: str, color='red'):
    x, y, w, h = box
    # text
    ax.text(x, y-5, label, fontsize=12, color='black', bbox={'facecolor': 'white', 'alpha': 0.4})
    # bbox
    rect = Rectangle((x, y), w, h, edgecolor=color, facecolor='none')
    ax.add_patch(rect)


def plot_img_from_region(d: dict, N):
    im_id = d['image_id']

    img = Image.open(id2paths[im_id])

    plt.figure(figsize=(14, 14))
    plt.imshow(img)

    # reference
    axes = plt.gca()

    for r in d['regions'][:N]:
        text = r['phrase']
        bbox = [r['x'], r['y'], r['width'], r['height']]

        _annotate(axes, bbox, text)

    plt.show()


def plot_img_from_attr(d: dict):
    im_id = d['image_id']

    img = Image.open(id2paths[im_id])

    plt.figure(figsize=(14, 14))
    plt.imshow(img)

    # reference
    axes = plt.gca()

    for r in d['attributes']:
        name = r['names']
        synset = r['synsets']

        obj = name[0] if len(synset) == 0 else synset[0]

        if 'attributes' in r:
            attr = r['attributes']

            text = f"{obj}: {attr}"
            bbox = [r['x'], r['y'], r['w'], r['h']]

            _annotate(axes, bbox, text)

    plt.show()

# Visualize Image + BBoxes

Set object font color (bbox region) to the model prediction (OFA).


In [7]:
# for sample in data_attr[100:200]:
#     plot_img_from_attr(sample)

####  Object → Images → Colors

In [8]:
def _read_txt(path) -> list:
    with open(path) as f:
        return f.read().split()


synsets = _read_txt('./temp/objects_100.txt')

In [9]:
from data_dev import ColorDataDev

color_dev = ColorDataDev(data_attr, device='cuda:0')

In [10]:
synset2images = color_dev.map_synset_to_images(synsets)

100%|██████████| 100/100 [00:39<00:00,  2.54it/s]


In [13]:
save_json(synset2images, path='./temp/objects_to_images_100.json')

### Regions Data

We work with subset (10k) of data.

In [11]:
data = read_json('./temp/region_graphs_10k.json')

In [12]:
# plot_img_from_region(data[24], N=10)